# Import Packages

In [29]:
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import json
import pandas as pd
from urllib.request import urlopen
from json import loads

# Define Helper Functions

In [30]:
def get_educationdata(first_url):
    all_data = [] # container for data
    
    # Grab data from first url
    response = urlopen(first_url)
    data = loads(response.read())
    count = len(data['results'])
    all_data.append(data['results'])
    i = len(all_data)

    print('expecting ' + str(data['count']) + ' rows of data')
    
    # For following 'next' urls grab data, append to container
    while count < data['count']:
        if data['next'] is not None:
            try:
                response = urlopen(data['next'])
                data = loads(response.read())
                count += len(data['results'])
                all_data.append(data['results'])
                i += 1
                print('appended page ' + str(i))
                
            except:
                print('failed to append page ' + str(i))
    return all_data

In [31]:
def pages_to_one_df(data):
    data_dfs = []
    for page in range(0, len(data)):
        data_dfs.append(pd.DataFrame(data[page]))

    return pd.concat(data_dfs).reset_index(drop = True)

In [32]:
def data_to_tuples(data):
    data_tup = []
    for row in data.values:
        data_tup.append(tuple(row))
    return data_tup

In [33]:
def insert_data(data_tup, tbl_name, columns):
    
    # Connect to database
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'colleges',
    buffered = True
    )
    cursor = cnx.cursor()
    
    # Create insert statement
    len_col = len(columns)
    col_names = ', '.join(columns)
    num_col = '%s, '*len_col
    insert_statement = f"INSERT INTO {tbl_name} ({col_names}) VALUES ({num_col.strip(', ')})"
    print(insert_statement)
    
    # Execute insert statement for each tuple in list of data
    cursor.executemany(insert_statement, data_tup)
    cnx.commit()
    
    # Close connection
    cnx.close()
    return 'inserted data'

# Get Directory Info for Degree-Granting Institutions

In [34]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/directory/2016"
ipeds_dir_data = get_educationdata(url)
ipeds_dir_df = pages_to_one_df(ipeds_dir_data)

expecting 7521 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8


In [35]:
ipeds_dir_df.shape

(7521, 76)

In [36]:
ipeds_dir_columns = ['unitid', 'year', 'inst_name', 'state_abbr', 'region',
                     'urban_centric_locale', 'inst_status', 'currently_active_ipeds', 
                     'degree_granting', 'open_public', 'title_iv_indicator', 
                     'postsec_public_active', 'postsec_public_active_title_iv', 
                     'year_deleted', 'inst_control', 'institution_level', 'inst_category', 
                     'inst_size', 'sector', 'primarily_postsecondary', 'hbcu', 
                     'hospital', 'medical_degree', 'tribal_college', 'land_grant', 
                     'offering_highest_degree', 'offering_highest_level', 'offering_undergrad', 
                     'offering_grad', 'cc_basic_2015', 'cc_instruc_undergrad_2015', 
                     'cc_enroll_2015', 'cc_size_setting_2015']
ipeds_dir = ipeds_dir_df[ipeds_dir_columns]

In [37]:
ipeds_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7521 entries, 0 to 7520
Data columns (total 33 columns):
unitid                            7521 non-null int64
year                              7521 non-null int64
inst_name                         7521 non-null object
state_abbr                        7521 non-null object
region                            7521 non-null int64
urban_centric_locale              7521 non-null int64
inst_status                       7521 non-null object
currently_active_ipeds            7521 non-null int64
degree_granting                   7521 non-null int64
open_public                       7521 non-null int64
title_iv_indicator                7521 non-null int64
postsec_public_active             7521 non-null int64
postsec_public_active_title_iv    7521 non-null int64
year_deleted                      7521 non-null int64
inst_control                      7521 non-null int64
institution_level                 7521 non-null int64
inst_category             

In [39]:
ipeds_dir = ipeds_dir.astype({'inst_status': 'int64'})
ipeds_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7521 entries, 0 to 7520
Data columns (total 33 columns):
unitid                            7521 non-null int64
year                              7521 non-null int64
inst_name                         7521 non-null object
state_abbr                        7521 non-null object
region                            7521 non-null int64
urban_centric_locale              7521 non-null int64
inst_status                       7521 non-null int64
currently_active_ipeds            7521 non-null int64
degree_granting                   7521 non-null int64
open_public                       7521 non-null int64
title_iv_indicator                7521 non-null int64
postsec_public_active             7521 non-null int64
postsec_public_active_title_iv    7521 non-null int64
year_deleted                      7521 non-null int64
inst_control                      7521 non-null int64
institution_level                 7521 non-null int64
inst_category              

In [40]:
ipeds_dir.inst_status.value_counts() # 1 = active institution
ipeds_dir = ipeds_dir[ipeds_dir['inst_status'] == 1]

In [41]:
ipeds_dir = ipeds_dir[ipeds_dir['offering_undergrad'] == 1]
ipeds_dir = ipeds_dir[ipeds_dir['degree_granting'] == 1]

In [42]:
ipeds_dir.region.value_counts()

5    1124
2     661
8     620
3     616
6     446
4     435
1     241
7     161
9      99
0       5
Name: region, dtype: int64

## Insert into MySQL Table

In [80]:
ipeds_dir_tuple = data_to_tuples(ipeds_dir)
insert_data(ipeds_dir_tuple, 'ipeds_dir', ipeds_dir_columns)

INSERT INTO ipeds_dir (unitid, year, inst_name, state_abbr, region, urban_centric_locale, inst_status, currently_active_ipeds, degree_granting, open_public, title_iv_indicator, postsec_public_active, postsec_public_active_title_iv, year_deleted, inst_control, institution_level, inst_category, inst_size, sector, primarily_postsecondary, hbcu, hospital, medical_degree, tribal_college, land_grant, offering_highest_degree, offering_highest_level, offering_undergrad, offering_grad, cc_basic_2015, cc_instruc_undergrad_2015, cc_enroll_2015, cc_size_setting_2015) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


'inserted data'

# Get Institutional Characteristics

In [81]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/institutional-characteristics/2016/"
ipeds_inst_char = get_educationdata(url)
ipeds_inst_char_df = pages_to_one_df(ipeds_inst_char)

expecting 7521 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8


In [97]:
ipeds_inst_char_columns = ['unitid', 'year', 'inst_affiliation', 'primary_public_control', 
                           'occupational_prog_offered', 'academic_prog_offered', 
                           'cont_prof_prog_offered', 'avocational_prog_offered', 
                           'remedial_prog_offered', 'secondary_prog_offered', 'assoc_offered', 
                           'bach_offered', 'postbac_cert_offered', 'masters_offered', 
                           'doctors_research_offered', 'doctors_professional_offered', 
                           'doctors_other_offered', 'enrolled_undergrad_fulltime', 
                           'remedial_services', 'academic_counseling', 'employment_services', 
                           'placement_services', 'oncampus_daycare', 'oncampus_housing', 
                           'oncampus_required', 'meal_plan', 'typical_room_charge', 
                           'typical_board_charge', 'room_board_charge', 'tuition_varies', 
                           'alt_tuition', 'religious_affiliation', 'ap_credit', 'rotc', 
                           'study_abroad', 'weekend_evening_college', 'disability_indicator', 
                           'yellow_ribbon_program', 'veteran_point_of_contact', 
                           'student_veteran_organization', 'no_vet_services', 'library_physical', 
                           'library_digital', 'library_trained_staff', 'library_shared', 
                           'no_library', 'calendar_system']
ipeds_inst_char = ipeds_inst_char_df[ipeds_inst_char_columns]

In [104]:
# Replace nan values with None
# Check if in directory
ipeds_inst_char = ipeds_inst_char.where((pd.notnull(ipeds_inst_char)), None)
ipeds_inst_char = ipeds_inst_char[ipeds_inst_char['unitid'].isin(ipeds_dir['unitid'])]

In [107]:
ipeds_inst_char.info() # looks like ~200 data points may be missing most data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 0 to 7339
Data columns (total 47 columns):
unitid                          4408 non-null int64
year                            4408 non-null int64
inst_affiliation                4261 non-null object
primary_public_control          4261 non-null object
occupational_prog_offered       4261 non-null object
academic_prog_offered           4261 non-null object
cont_prof_prog_offered          4261 non-null object
avocational_prog_offered        4261 non-null object
remedial_prog_offered           4261 non-null object
secondary_prog_offered          4261 non-null object
assoc_offered                   4261 non-null object
bach_offered                    4261 non-null object
postbac_cert_offered            4261 non-null object
masters_offered                 4261 non-null object
doctors_research_offered        4261 non-null object
doctors_professional_offered    4261 non-null object
doctors_other_offered           4261 non-null o

## Insert into MySql Table

In [112]:
ipeds_inst_char_tuple = data_to_tuples(ipeds_inst_char)
insert_data(ipeds_inst_char_tuple, 'ipeds_inst_char', ipeds_inst_char_columns)

INSERT INTO ipeds_inst_char (unitid, year, inst_affiliation, primary_public_control, occupational_prog_offered, academic_prog_offered, cont_prof_prog_offered, avocational_prog_offered, remedial_prog_offered, secondary_prog_offered, assoc_offered, bach_offered, postbac_cert_offered, masters_offered, doctors_research_offered, doctors_professional_offered, doctors_other_offered, enrolled_undergrad_fulltime, remedial_services, academic_counseling, employment_services, placement_services, oncampus_daycare, oncampus_housing, oncampus_required, meal_plan, typical_room_charge, typical_board_charge, room_board_charge, tuition_varies, alt_tuition, religious_affiliation, ap_credit, rotc, study_abroad, weekend_evening_college, disability_indicator, yellow_ribbon_program, veteran_point_of_contact, student_veteran_organization, no_vet_services, library_physical, library_digital, library_trained_staff, library_shared, no_library, calendar_system) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s

'inserted data'

# Get Enrollment Data

In [223]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/admissions-enrollment/2016/"
ipeds_adm_enroll = get_educationdata(url)
ipeds_adm_enroll_df = pages_to_one_df(ipeds_adm_enroll)

expecting 22563 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8
appended page 9
appended page 10
appended page 11
appended page 12
appended page 13
appended page 14
appended page 15
appended page 16
appended page 17
appended page 18
appended page 19
appended page 20
appended page 21
appended page 22
appended page 23


In [224]:
# Replace numerical sex var with string representation
ipeds_adm_enroll_df = ipeds_adm_enroll_df.drop(['fips', 'year'], axis = 1)
ipeds_adm_enroll_df = ipeds_adm_enroll_df.replace({'sex': {1: 'M', 2: 'F', 99: 'N'}})

# Pivot dataframe
ipeds_adm_enroll_piv = ipeds_adm_enroll_df.pivot(index = 'unitid', columns = 'sex')
ipeds_adm_enroll_piv.reset_index(inplace = True)
ipeds_adm_enroll_piv.columns = ['_'.join(col).strip('_') for col in ipeds_adm_enroll_piv.columns.values]

In [225]:
# Replace nan values with None
# Check if in directory
ipeds_adm_enroll_piv = ipeds_adm_enroll_piv.where((pd.notnull(ipeds_adm_enroll_piv)), None)
ipeds_adm_enroll_piv = ipeds_adm_enroll_piv[ipeds_adm_enroll_piv['unitid'].isin(ipeds_dir['unitid'])] 

In [226]:
ipeds_adm_enroll_piv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408 entries, 0 to 7339
Data columns (total 16 columns):
unitid                     4408 non-null int64
number_admitted_F          1833 non-null object
number_admitted_M          1849 non-null object
number_admitted_N          1897 non-null object
number_applied_F           1906 non-null object
number_applied_M           1906 non-null object
number_applied_N           1907 non-null object
number_enrolled_ft_F       1822 non-null object
number_enrolled_ft_M       1832 non-null object
number_enrolled_ft_N       1892 non-null object
number_enrolled_pt_F       1479 non-null object
number_enrolled_pt_M       1444 non-null object
number_enrolled_pt_N       1490 non-null object
number_enrolled_total_F    1830 non-null object
number_enrolled_total_M    1840 non-null object
number_enrolled_total_N    1895 non-null object
dtypes: int64(1), object(15)
memory usage: 585.4+ KB


## Insert into MySQL Table

In [228]:
ipeds_adm_enroll_tuple = data_to_tuples(ipeds_adm_enroll_piv)
insert_data(ipeds_adm_enroll_tuple, 'ipeds_adm_enroll', ipeds_adm_enroll_piv.columns)

INSERT INTO ipeds_adm_enroll (unitid, number_admitted_F, number_admitted_M, number_admitted_N, number_applied_F, number_applied_M, number_applied_N, number_enrolled_ft_F, number_enrolled_ft_M, number_enrolled_ft_N, number_enrolled_pt_F, number_enrolled_pt_M, number_enrolled_pt_N, number_enrolled_total_F, number_enrolled_total_M, number_enrolled_total_N) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


'inserted data'

# Get Scorecard Student Characteristics

In [206]:
url = "https://educationdata.urban.org/api/v1/college-university/scorecard/student-characteristics/2016/aid-applicants/"
sc_stud_char = get_educationdata(url)
sc_stud_char_df = pages_to_one_df(sc_stud_char)

expecting 6976 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7


In [235]:
sc_stud_char_df.describe()

,count_family_income,count_family_income_dependent,count_family_income_independent,count_parent_ed,count_total_fafsa_applicants,count_valid_dep_status,dependent_highincome1_pct,dependent_highincome2_pct,dependent_highincome_pct,dependent_income_mean,...,midincome1_pct,midincome2_pct,midincome_pct,opeid6,parents_highest_ed_high_school,parents_highest_ed_middle_school,parents_highest_ed_postsecondary,unitid,veteran_pct,year
count,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,6976.000000,1921.000000,6976.000000,...,6976.000000,6976.000000,5121.000000,6976.000000,6976.000000,6976.000000,6976.000000,6.976000e+03,6976.000000,6976.0
mean,3159.814220,1230.813217,1922.176892,2622.659404,3159.814220,3159.814220,-2.129866,-2.132989,0.254190,45254.752133,...,-0.368712,-0.668607,0.273947,16546.096474,-1.001577,-1.006737,-0.399943,1.852134e+06,-1.851927,2016.0
std,7091.258552,2442.708313,5849.503324,5873.126193,7091.258552,7091.258552,1.399859,1.403510,0.147374,26352.561264,...,1.173290,1.356935,0.064493,14369.921477,1.414871,1.416679,1.019815,6.932749e+06,1.464363,0.0
min,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,-3.000000,0.019750,-3.000000,...,-3.000000,-3.000000,0.065241,1002.000000,-3.000000,-3.000000,-3.000000,1.006540e+05,-3.000000,2016.0
25%,184.000000,55.000000,73.750000,144.000000,184.000000,184.000000,-3.000000,-3.000000,0.145604,27790.700000,...,0.106136,-3.000000,0.231626,3239.000000,-3.000000,-3.000000,0.000000,1.741248e+05,-3.000000,2016.0
50%,771.000000,391.000000,281.000000,650.500000,771.000000,771.000000,-3.000000,-3.000000,0.218125,42391.250000,...,0.145091,0.098183,0.281307,10542.000000,0.000000,0.000000,0.000000,2.290495e+05,-3.000000,2016.0
75%,2661.250000,1345.250000,1129.750000,2180.250000,2661.250000,2661.250000,0.043916,0.038351,0.354872,60715.800000,...,0.172916,0.141916,0.319785,26216.250000,0.000000,0.000000,0.000000,4.495985e+05,0.008010,2016.0
max,61975.000000,22902.000000,56938.000000,50030.000000,61975.000000,61975.000000,0.500000,0.559146,0.694932,152069.000000,...,0.434783,0.354839,0.612179,42580.000000,0.000000,0.000000,0.000000,4.877820e+07,0.346939,2016.0


In [213]:
sc_stud_char_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6976 entries, 0 to 6975
Data columns (total 51 columns):
age_24orolder_pct                   0 non-null object
count_family_income                 6976 non-null int64
count_family_income_dependent       6976 non-null int64
count_family_income_independent     6976 non-null int64
count_parent_ed                     6976 non-null int64
count_total_fafsa_applicants        6976 non-null int64
count_valid_dep_status              6976 non-null int64
dependent_highincome1_pct           6976 non-null float64
dependent_highincome2_pct           6976 non-null float64
dependent_highincome_pct            1921 non-null float64
dependent_income_mean               6976 non-null float64
dependent_lowincome_pct             6976 non-null float64
dependent_midincome1_pct            6976 non-null float64
dependent_midincome2_pct            6976 non-null float64
dependent_midincome_pct             3263 non-null float64
dependent_pct                       697

In [212]:
sc_stud_char_df.columns

Index(['age_24orolder_pct', 'count_family_income',
       'count_family_income_dependent', 'count_family_income_independent',
       'count_parent_ed', 'count_total_fafsa_applicants',
       'count_valid_dep_status', 'dependent_highincome1_pct',
       'dependent_highincome2_pct', 'dependent_highincome_pct',
       'dependent_income_mean', 'dependent_lowincome_pct',
       'dependent_midincome1_pct', 'dependent_midincome2_pct',
       'dependent_midincome_pct', 'dependent_pct', 'fafsa_sent_2ormore_pct',
       'fafsa_sent_3ormore_pct', 'fafsa_sent_4ormore_pct',
       'fafsa_sent_5ormore_pct', 'faminc_independents_mean', 'faminc_mean',
       'faminc_med', 'female_pct', 'fips', 'first_gen_student_pct',
       'highincome1_pct', 'highincome2_pct', 'highincome_pct',
       'independent_highincome1_pct', 'independent_highincome2_pct',
       'independent_highincome_pct', 'independent_income_mean',
       'independent_lowincome_pct', 'independent_midincome1_pct',
       'independent_midinc

In [236]:
sc_stud_char_columns = ['unitid', 'lowincome_pct', 'independent_pct', 
                        'dependent_lowincome_pct', 'independent_lowincome_pct', 
                        'midincome1_pct', 'midincome2_pct', 'highincome1_pct', 
                        'highincome2_pct', 'dependent_midincome1_pct', 
                        'dependent_midincome2_pct', 'dependent_highincome1_pct', 
                        'dependent_highincome2_pct', 'independent_midincome1_pct', 
                        'independent_midincome2_pct', 'independent_highincome1_pct', 
                        'independent_highincome2_pct', 'fafsa_sent_2ormore_pct', 
                        'fafsa_sent_3ormore_pct', 'fafsa_sent_4ormore_pct', 
                        'fafsa_sent_5ormore_pct', 'dependent_income_mean', 
                        'independent_income_mean', 'count_family_income', 
                        'count_family_income_dependent', 'count_family_income_independent', 
                        'count_valid_dep_status', 'count_parent_ed', 
                        'count_total_fafsa_applicants', 'female_pct', 'married_pct', 
                        'dependent_pct', 'veteran_pct', 'first_gen_student_pct', 
                        'faminc_mean', 'faminc_med', 'faminc_independents_mean', 
                        'midincome_pct', 'highincome_pct', 'dependent_midincome_pct', 
                        'dependent_highincome_pct', 'independent_midincome_pct', 
                        'independent_highincome_pct']
sc_stud_char = sc_stud_char_df[sc_stud_char_columns]

# Replace nan values with None
# Check if in directory
sc_stud_char = sc_stud_char.where((pd.notnull(sc_stud_char)), None)
sc_stud_char = sc_stud_char[sc_stud_char['unitid'].isin(ipeds_dir['unitid'])]

## Insert into MySQL Table

In [239]:
sc_stud_char_tuple = data_to_tuples(sc_stud_char)
insert_data(sc_stud_char_tuple, 'sc_stud_char', sc_stud_char.columns)

INSERT INTO sc_stud_char (unitid, lowincome_pct, independent_pct, dependent_lowincome_pct, independent_lowincome_pct, midincome1_pct, midincome2_pct, highincome1_pct, highincome2_pct, dependent_midincome1_pct, dependent_midincome2_pct, dependent_highincome1_pct, dependent_highincome2_pct, independent_midincome1_pct, independent_midincome2_pct, independent_highincome1_pct, independent_highincome2_pct, fafsa_sent_2ormore_pct, fafsa_sent_3ormore_pct, fafsa_sent_4ormore_pct, fafsa_sent_5ormore_pct, dependent_income_mean, independent_income_mean, count_family_income, count_family_income_dependent, count_family_income_independent, count_valid_dep_status, count_parent_ed, count_total_fafsa_applicants, female_pct, married_pct, dependent_pct, veteran_pct, first_gen_student_pct, faminc_mean, faminc_med, faminc_independents_mean, midincome_pct, highincome_pct, dependent_midincome_pct, dependent_highincome_pct, independent_midincome_pct, independent_highincome_pct) VALUES (%s, %s, %s, %s, %s, %s, 

'inserted data'

# Get Enrollment Headcount

In [7]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/enrollment-headcount/2016/1/"
ipeds_ug_headcount = get_educationdata(url)
ipeds_ug_headcount_df = pages_to_one_df(ipeds_ug_headcount)

expecting 194190 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8
appended page 9
appended page 10
appended page 11
appended page 12
appended page 13
appended page 14
appended page 15
appended page 16
appended page 17
appended page 18
appended page 19
appended page 20
appended page 21
appended page 22
appended page 23
appended page 24
appended page 25
appended page 26
appended page 27
appended page 28
appended page 29
appended page 30
appended page 31
appended page 32
appended page 33
appended page 34
appended page 35
appended page 36
appended page 37
appended page 38
appended page 39
appended page 40
appended page 41
appended page 42
appended page 43
appended page 44
appended page 45
appended page 46
appended page 47
appended page 48
appended page 49
appended page 50
appended page 51
appended page 52
appended page 53
appended page 54
appended page 55
appended page 56
appended page 57
appended page 58
appended 

In [19]:
ipeds_ug_headcount_df = ipeds_ug_headcount_df.drop(['fips', 'year'], axis = 1)

In [43]:
# Replace nan values with None
# Check if in directory
ipeds_ug_headcount_df = ipeds_ug_headcount_df.where((pd.notnull(ipeds_ug_headcount_df)), None)
ipeds_ug_headcount_df = ipeds_ug_headcount_df[ipeds_ug_headcount_df['unitid'].isin(ipeds_dir['unitid'])] 

In [44]:
ipeds_ug_headcount_df.shape

(123060, 5)

In [46]:
ipeds_ug_headcount_df.replace({'sex': {1: 'M', 2: 'F', 9: 'U', 99: 'Total', 
                                       -1: 'missing', -2: 'NA', -3: 'suppressed'}, 
                               'race': {1: 'wht', 2: 'blk', 3: 'hsp', 4: 'asn', 
                                        5: 'ntv', 6: 'hwn/pi', 7: 'mlt', 8: 'nonres', 
                                        9: 'unknown', 20: 'other', 99: 'total', 
                                        -1: 'missing', -2: 'NA', -3: 'suppressed'}}, 
                              inplace = True)

In [59]:
ipeds_ug_headcount_piv = pd.pivot_table(ipeds_ug_headcount_df, index = 'unitid', columns = ['race', 'sex']).reset_index()

In [60]:
columns = ['_'.join(col).strip('_') for col in ipeds_ug_headcount_piv.columns.values]
ipeds_ug_headcount_piv.columns = columns

In [61]:
ipeds_ug_headcount_piv.head()

,unitid,headcount_asn_F,headcount_asn_M,headcount_asn_Total,headcount_blk_F,headcount_blk_M,headcount_blk_Total,headcount_hsp_F,headcount_hsp_M,headcount_hsp_Total,...,level_of_study_ntv_Total,level_of_study_total_F,level_of_study_total_M,level_of_study_total_Total,level_of_study_unknown_F,level_of_study_unknown_M,level_of_study_unknown_Total,level_of_study_wht_F,level_of_study_wht_M,level_of_study_wht_Total
0,100654,6,6,12,2679,2063,4742,18,18,36,...,1,1,1,1,1,1,1,1,1,1
1,100663,417,326,743,2355,1142,3497,243,186,429,...,1,1,1,1,1,1,1,1,1,1
2,100690,1,1,2,139,56,195,25,19,44,...,1,1,1,1,1,1,1,1,1,1
3,100706,137,167,304,463,350,813,143,170,313,...,1,1,1,1,1,1,1,1,1,1
4,100724,9,8,17,3013,1854,4867,30,29,59,...,1,1,1,1,1,1,1,1,1,1


In [62]:
ipeds_ug_headcount_piv.shape

(4102, 61)

In [63]:
ipeds_ug_headcount_piv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4102 entries, 0 to 4101
Data columns (total 61 columns):
unitid                          4102 non-null int64
headcount_asn_F                 4102 non-null int64
headcount_asn_M                 4102 non-null int64
headcount_asn_Total             4102 non-null int64
headcount_blk_F                 4102 non-null int64
headcount_blk_M                 4102 non-null int64
headcount_blk_Total             4102 non-null int64
headcount_hsp_F                 4102 non-null int64
headcount_hsp_M                 4102 non-null int64
headcount_hsp_Total             4102 non-null int64
headcount_hwn/pi_F              4102 non-null int64
headcount_hwn/pi_M              4102 non-null int64
headcount_hwn/pi_Total          4102 non-null int64
headcount_mlt_F                 4102 non-null int64
headcount_mlt_M                 4102 non-null int64
headcount_mlt_Total             4102 non-null int64
headcount_nonres_F              4102 non-null int64
headcount

In [76]:
# All level_of_study is 1 --> columns are irrelevant
pd.DataFrame(ipeds_ug_headcount_piv.describe()).iloc[:, 50:]

,level_of_study_ntv_M,level_of_study_ntv_Total,level_of_study_total_F,level_of_study_total_M,level_of_study_total_Total,level_of_study_unknown_F,level_of_study_unknown_M,level_of_study_unknown_Total,level_of_study_wht_F,level_of_study_wht_M,level_of_study_wht_Total
count,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0,4102.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [126]:
cols = ['unitid'] + [x for x in ipeds_ug_headcount_piv.columns if 'headcount' in x]

In [127]:
cols

['unitid',
 'headcount_asn_F',
 'headcount_asn_M',
 'headcount_asn_Total',
 'headcount_blk_F',
 'headcount_blk_M',
 'headcount_blk_Total',
 'headcount_hsp_F',
 'headcount_hsp_M',
 'headcount_hsp_Total',
 'headcount_hwn/pi_F',
 'headcount_hwn/pi_M',
 'headcount_hwn/pi_Total',
 'headcount_mlt_F',
 'headcount_mlt_M',
 'headcount_mlt_Total',
 'headcount_nonres_F',
 'headcount_nonres_M',
 'headcount_nonres_Total',
 'headcount_ntv_F',
 'headcount_ntv_M',
 'headcount_ntv_Total',
 'headcount_total_F',
 'headcount_total_M',
 'headcount_total_Total',
 'headcount_unknown_F',
 'headcount_unknown_M',
 'headcount_unknown_Total',
 'headcount_wht_F',
 'headcount_wht_M',
 'headcount_wht_Total']

In [128]:
ipeds_ug_headcount_sub = ipeds_ug_headcount_piv[cols]
ipeds_ug_headcount_sub.columns = [x.replace('/', '') for x in cols]

In [129]:
ipeds_ug_headcount_sub.head()

,unitid,headcount_asn_F,headcount_asn_M,headcount_asn_Total,headcount_blk_F,headcount_blk_M,headcount_blk_Total,headcount_hsp_F,headcount_hsp_M,headcount_hsp_Total,...,headcount_ntv_Total,headcount_total_F,headcount_total_M,headcount_total_Total,headcount_unknown_F,headcount_unknown_M,headcount_unknown_Total,headcount_wht_F,headcount_wht_M,headcount_wht_Total
0,100654,6,6,12,2679,2063,4742,18,18,36,...,11,2921,2286,5207,7,4,11,55,82,137
1,100663,417,326,743,2355,1142,3497,243,186,429,...,36,7780,5528,13308,109,89,198,4188,3327,7515
2,100690,1,1,2,139,56,195,25,19,44,...,0,254,177,431,59,64,123,29,37,66
3,100706,137,167,304,463,350,813,143,170,313,...,90,3232,4287,7519,293,302,595,2021,3034,5055
4,100724,9,8,17,3013,1854,4867,30,29,59,...,8,3205,2024,5229,29,22,51,41,42,83


In [130]:
ipeds_ug_headcount_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4102 entries, 0 to 4101
Data columns (total 31 columns):
unitid                     4102 non-null int64
headcount_asn_F            4102 non-null int64
headcount_asn_M            4102 non-null int64
headcount_asn_Total        4102 non-null int64
headcount_blk_F            4102 non-null int64
headcount_blk_M            4102 non-null int64
headcount_blk_Total        4102 non-null int64
headcount_hsp_F            4102 non-null int64
headcount_hsp_M            4102 non-null int64
headcount_hsp_Total        4102 non-null int64
headcount_hwnpi_F          4102 non-null int64
headcount_hwnpi_M          4102 non-null int64
headcount_hwnpi_Total      4102 non-null int64
headcount_mlt_F            4102 non-null int64
headcount_mlt_M            4102 non-null int64
headcount_mlt_Total        4102 non-null int64
headcount_nonres_F         4102 non-null int64
headcount_nonres_M         4102 non-null int64
headcount_nonres_Total     4102 non-null int64


## Insert into MySQL

In [135]:
# For each value, convert to Python native int type (np int type not acceptable to MySQL)
ipeds_ug_headcount_tuple = []
for row in ipeds_ug_headcount_sub.values:
    row_lst = []
    for item in row:
        row_lst.append(item.item())
    ipeds_ug_headcount_tuple.append(tuple(row_lst))

In [132]:
ipeds_ug_headcount_tuple[0]

(100654,
 6,
 6,
 12,
 2679,
 2063,
 4742,
 18,
 18,
 36,
 5,
 5,
 10,
 128,
 82,
 210,
 15,
 23,
 38,
 8,
 3,
 11,
 2921,
 2286,
 5207,
 7,
 4,
 11,
 55,
 82,
 137)

In [134]:
# ipeds_ug_headcount_tuple = data_to_tuples(ipeds_ug_headcount_sub)
insert_data(ipeds_ug_headcount_tuple, 'ipeds_ug_headcount', ipeds_ug_headcount_sub.columns)

INSERT INTO ipeds_ug_headcount (unitid, headcount_asn_F, headcount_asn_M, headcount_asn_Total, headcount_blk_F, headcount_blk_M, headcount_blk_Total, headcount_hsp_F, headcount_hsp_M, headcount_hsp_Total, headcount_hwnpi_F, headcount_hwnpi_M, headcount_hwnpi_Total, headcount_mlt_F, headcount_mlt_M, headcount_mlt_Total, headcount_nonres_F, headcount_nonres_M, headcount_nonres_Total, headcount_ntv_F, headcount_ntv_M, headcount_ntv_Total, headcount_total_F, headcount_total_M, headcount_total_Total, headcount_unknown_F, headcount_unknown_M, headcount_unknown_Total, headcount_wht_F, headcount_wht_M, headcount_wht_Total) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


'inserted data'

# Get Graduation Rate Info Based on Financial Aid

In [29]:
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/grad-rates-pell/2016/"
ipeds_grad_fa = get_educationdata(url)
ipeds_grad_fa_df = pages_to_one_df(ipeds_grad_fa)

expecting 43228 rows of data
appended page 2
appended page 3
appended page 4
appended page 5
appended page 6
appended page 7
appended page 8
appended page 9
appended page 10
appended page 11
appended page 12
appended page 13
appended page 14
appended page 15
appended page 16
appended page 17
appended page 18
appended page 19
appended page 20
appended page 21
appended page 22
appended page 23
appended page 24
appended page 25
appended page 26
appended page 27
appended page 28
appended page 29
appended page 30
appended page 31
appended page 32
appended page 33
appended page 34
appended page 35
appended page 36
appended page 37
appended page 38
appended page 39
appended page 40
appended page 41
appended page 42
appended page 43
appended page 44


In [30]:
ipeds_grad_fa_df.head()

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,fips,institution_level,subcohort,unitid,year
0,24,24,2011,12,0.500,0,99,17,4,2,147536,2016
1,4,4,2011,0,0.000,0,3,27,4,99,440767,2016
2,137,137,2014,100,0.730,0,1,6,1,99,119553,2016
3,8,8,2014,8,1.000,0,99,9,3,99,417178,2016
4,139,139,2011,40,0.288,0,3,42,4,99,215275,2016


In [31]:
ipeds_grad_fa_df.describe()

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,fips,institution_level,subcohort,unitid,year
count,43228.000000,43228.000000,43228.000000,43228.000000,39301.000000,43228.000000,43228.00000,43228.000000,43228.000000,43228.000000,43228.000000,43228.0
mean,219.185435,220.044740,2012.457389,115.157444,0.543069,0.859304,26.25000,29.774591,3.201258,70.497085,276042.314518,2016.0
std,696.282169,699.116868,1.499412,350.848093,0.277793,21.439059,42.00867,16.796727,1.048625,44.291262,132209.978858,0.0
min,0.000000,0.000000,2011.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000,1.000000,100654.000000,2016.0
25%,7.000000,7.000000,2011.000000,3.000000,0.333000,0.000000,1.75000,15.000000,3.000000,2.000000,168342.000000,2016.0
50%,44.000000,44.000000,2011.000000,22.000000,0.553000,0.000000,2.50000,31.000000,4.000000,99.000000,217624.000000,2016.0
75%,187.000000,187.000000,2014.000000,89.000000,0.750000,0.000000,27.00000,42.000000,4.000000,99.000000,438285.000000,2016.0
max,50695.000000,50789.000000,2014.000000,13600.000000,1.000000,2131.000000,99.00000,78.000000,4.000000,99.000000,491242.000000,2016.0


In [32]:
# Replace numerical sex var with string representation
ipeds_grad_fa_df = ipeds_grad_fa_df.drop(['fips', 'year'], axis = 1)
ipeds_grad_fa_df = ipeds_grad_fa_df.replace({'subcohort': {1: 'nonbach', 2: 'bach', 99: 'total'}})
ipeds_grad_fa_df = ipeds_grad_fa_df.replace({'fed_aid_type': {1: 'pell', 2: 'stafford', 3: 'neither', 4: 'not_pell', 99: 'total'}})

ipeds_grad_fa_df.head()

,cohort_adj,cohort_rev,cohort_year,completers_150pct,completion_rate_150pct,exclusions,fed_aid_type,institution_level,subcohort,unitid
0,24,24,2011,12,0.500,0,total,4,bach,147536
1,4,4,2011,0,0.000,0,neither,4,total,440767
2,137,137,2014,100,0.730,0,pell,1,total,119553
3,8,8,2014,8,1.000,0,total,3,total,417178
4,139,139,2011,40,0.288,0,neither,4,total,215275


## Get Data for Bachelor's Degrees

In [33]:
# Subset for bachelor's degrees, pivot table
ipeds_grad_fa_bach = ipeds_grad_fa_df[ipeds_grad_fa_df['subcohort'] == 'bach']
ipeds_grad_fa_bach = ipeds_grad_fa_bach.pivot(index = 'unitid', columns = 'fed_aid_type')
ipeds_grad_fa_bach.reset_index(inplace = True)
ipeds_grad_fa_bach.columns = ['_'.join(col).strip('_') for col in ipeds_grad_fa_bach.columns.values]

In [34]:
# Replace nan values with None
# Check if in directory
ipeds_grad_fa_bach = ipeds_grad_fa_bach.where((pd.notnull(ipeds_grad_fa_bach)), None)
ipeds_grad_fa_bach = ipeds_grad_fa_bach[ipeds_grad_fa_bach['unitid'].isin(ipeds_dir['unitid'])] 

In [35]:
ipeds_grad_fa_bach.head()

,unitid,cohort_adj_neither,cohort_adj_pell,cohort_adj_stafford,cohort_adj_total,cohort_rev_neither,cohort_rev_pell,cohort_rev_stafford,cohort_rev_total,cohort_year_neither,...,exclusions_stafford,exclusions_total,institution_level_neither,institution_level_pell,institution_level_stafford,institution_level_total,subcohort_neither,subcohort_pell,subcohort_stafford,subcohort_total
0,100654,60,667,112,839,60,667,112,839,2011,...,0,0,4,4,4,4,bach,bach,bach,bach
1,100663,794,582,200,1576,795,582,201,1578,2011,...,1,2,4,4,4,4,bach,bach,bach,bach
2,100690,9,0,0,9,9,0,0,9,2011,...,0,0,4,4,4,4,bach,bach,bach,bach
3,100706,316,205,131,652,316,205,131,652,2011,...,0,0,4,4,4,4,bach,bach,bach,bach
4,100724,143,891,64,1098,143,891,64,1098,2011,...,0,0,4,4,4,4,bach,bach,bach,bach


In [41]:
ipeds_grad_fa_bach.shape

(2133, 33)

## Get Data for Nonbachelor's Degrees

In [36]:
# Subset for nonbachelor's degrees, pivot table
ipeds_grad_fa_nonbach = ipeds_grad_fa_df[ipeds_grad_fa_df['subcohort'] == 'nonbach']
ipeds_grad_fa_nonbach = ipeds_grad_fa_nonbach.pivot(index = 'unitid', columns = 'fed_aid_type')
ipeds_grad_fa_nonbach.reset_index(inplace = True)
ipeds_grad_fa_nonbach.columns = ['_'.join(col).strip('_') for col in ipeds_grad_fa_nonbach.columns.values]

In [37]:
# Replace nan values with None
# Check if in directory
ipeds_grad_fa_nonbach = ipeds_grad_fa_nonbach.where((pd.notnull(ipeds_grad_fa_nonbach)), None)
ipeds_grad_fa_nonbach = ipeds_grad_fa_nonbach[ipeds_grad_fa_nonbach['unitid'].isin(ipeds_dir['unitid'])] 

In [38]:
ipeds_grad_fa_nonbach.head()

,unitid,cohort_adj_neither,cohort_adj_pell,cohort_adj_stafford,cohort_adj_total,cohort_rev_neither,cohort_rev_pell,cohort_rev_stafford,cohort_rev_total,cohort_year_neither,...,exclusions_stafford,exclusions_total,institution_level_neither,institution_level_pell,institution_level_stafford,institution_level_total,subcohort_neither,subcohort_pell,subcohort_stafford,subcohort_total
0,100690,2,0,0,2,2,0,0,2,2011,...,0,0,4,4,4,4,nonbach,nonbach,nonbach,nonbach
1,101073,4,5,1,10,4,5,1,10,2011,...,0,0,4,4,4,4,nonbach,nonbach,nonbach,nonbach
2,101116,7,16,3,26,7,16,3,26,2011,...,0,0,4,4,4,4,nonbach,nonbach,nonbach,nonbach
3,101365,2,17,2,21,2,17,2,21,2011,...,0,0,4,4,4,4,nonbach,nonbach,nonbach,nonbach
4,101453,0,1,0,1,0,1,0,1,2011,...,0,0,4,4,4,4,nonbach,nonbach,nonbach,nonbach


In [39]:
ipeds_grad_fa_nonbach.shape

(1025, 33)

## Get Data for Total

In [54]:
# Subset for total, pivot table
ipeds_grad_fa_total = ipeds_grad_fa_df[ipeds_grad_fa_df['subcohort'] == 'total']
ipeds_grad_fa_total = ipeds_grad_fa_total.pivot(index = 'unitid', columns = ['fed_aid_type'])
#ipeds_grad_fa_total.reset_index(inplace = True)
#ipeds_grad_fa_total.columns = ['_'.join(col).strip('_') for col in ipeds_grad_fa_total.columns.values]

ValueError: all arrays must be same length

In [50]:
ipeds_dup = ipeds_grad_fa_total[ipeds_grad_fa_total.duplicated(subset = 'unitid')]